In [1]:
import pandas as pd
import numpy as np

In [2]:
entprise_train = pd.read_csv('train/entprise_info.csv',engine='python')
base_info = pd.read_csv('train/base_info.csv',engine='python',encoding='utf-8')
entprise_test = pd.read_csv('entprise_evaluate.csv')

In [3]:
entprise = pd.concat([entprise_train,entprise_test])
data = pd.merge(entprise,base_info,on='id')
data.drop(['ptbusscope','midpreindcode','score'],axis=1,inplace=True)

In [4]:
def opto_process(x):
    if len(x)>3:
        return x[:4]
    else:
        return x
data['opfrom'] = data['opfrom'].map(lambda x:x[:4])
data['opto'].fillna('nan',inplace=True)
data['opto'] = data['opto'].map(opto_process)
data['opfrom'] = data['opfrom'].astype(int)
data['opto'] = data['opto'].astype(float)
data['opform'] = data['opform'].fillna("货币")

In [5]:
train = data[data['enttypeitem'].notnull()]
test = data[data['enttypeitem'].isnull()]

In [6]:
train['enttypeitem_label'] = (train['enttypeitem']-train['enttype'])/10

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
x_train = train.drop(['id','enttypeitem_label','enttypeitem'],axis=1)
y_train = train['enttypeitem_label']
x_test = test.drop(['id','enttypeitem'],axis=1)

In [8]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8214 entries, 1 to 24859
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   label          4637 non-null   float64
 1   oplocdistrict  8214 non-null   int64  
 2   industryphy    8214 non-null   object 
 3   industryco     8214 non-null   float64
 4   dom            8214 non-null   object 
 5   opscope        8214 non-null   object 
 6   enttype        8214 non-null   int64  
 7   opfrom         8214 non-null   int32  
 8   opto           150 non-null    float64
 9   state          8214 non-null   int64  
 10  orgid          8214 non-null   int64  
 11  jobid          8214 non-null   int64  
 12  adbusign       8214 non-null   int64  
 13  townsign       8214 non-null   int64  
 14  regtype        8214 non-null   int64  
 15  empnum         8206 non-null   float64
 16  compform       8178 non-null   float64
 17  parnum         0 non-null      float64
 18  exenum 

In [9]:
from catboost import CatBoostClassifier
catboost_model = CatBoostClassifier(
    iterations=2000,
    od_type='Iter',
    od_wait=120,
    max_depth=8,
    learning_rate=0.02,
    l2_leaf_reg=9,
    random_seed=2020,
    metric_period=50,
    fold_len_multiplier=1.1,
    loss_function='MultiClass',
    logging_level='Verbose'
    )
cat_attrs = [2,4,5,19,23]
catboost_model.fit(x_train,y_train,cat_features=cat_attrs,verbose=500)
y_pred = catboost_model.predict(x_test)


0:	learn: 2.0525938	total: 242ms	remaining: 8m 4s
500:	learn: 0.0125408	total: 1m 18s	remaining: 3m 55s
1000:	learn: 0.0084459	total: 2m 41s	remaining: 2m 40s
1500:	learn: 0.0066994	total: 4m 4s	remaining: 1m 21s
1999:	learn: 0.0056082	total: 5m 29s	remaining: 0us


In [14]:
x_test['enttypeitem_label'] = y_pred

In [16]:
y_train.value_counts()

3.0    8967
5.0    5721
9.0     868
4.0     764
1.0     147
2.0     110
0.0      71
8.0       2
6.0       1
Name: enttypeitem_label, dtype: int64

In [15]:
x_test['enttypeitem_label'].value_counts()

0.0    8120
1.0      72
4.0      20
3.0       2
Name: enttypeitem_label, dtype: int64